In [1]:
import jieba
import re

def jieba_cut(_str):
    return ' '.join(jieba.lcut(_str))
def cnword_and_letter(_str):
    '''
    func: 文本清理，保留字符串中中文以及字母
    param: _str
        type: str
        detail
    return cleaned_str
        type: str
        detail: 经过strip去空格
    '''
    return "".join(re.findall("[\u4e00-\u9fa5a-zA-Z]", _str)).strip()

In [2]:
import pandas as pd

data_path = "./"

data = pd.read_csv(data_path + "train.csv")
data["question1"] = data["question1"].apply(cnword_and_letter)
data["question1"] = data["question1"].apply(jieba_cut)
data["question2"] = data["question2"].apply(cnword_and_letter)
data["question2"] = data["question2"].apply(jieba_cut)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 1.363 seconds.
Prefix dict has been built succesfully.


In [3]:
import gensim
import numpy as np



def get_w2v(splited_corpus, w2v_size, min_count):
    '''
    func: 获取word2vec模型
    param: splited_corpus
        type: pd.Series
        detail: 应当为训练集中所有语料
    param: w2v_size
        type: int
        detail: w2v向量维度
    return: w2v_model
        type: gensim.models.Word2Vec
        detail: 训练的模型只可以使用其transform接口
    '''
    sentences = [x.split() for x in splited_corpus]
    model = gensim.models.Word2Vec(sentences, min_count=min_count, size=w2v_size)
    return model

def get_w2v_key_vev(w2v_model):
    vecs = []
    words = []
    for word in w2v_model.wv.vocab:
        vecs.append(w2v_model[word])
        words.append(word)
    return words, vecs

def get_x_index(x, words):
    res = []
    for inst in x:
        res.append(np.array([words.index(word) for word in inst.split() if word in words]))
    return res

def mean_len(list_2d):
    meanlen = 0
    for arr in list_2d:
        meanlen += len(arr)
    return meanlen / len(list_2d)

def ceil2(num):
    res = 2
    while res < num:
        res *= 2
    return res

def padding(data2d, max_len, pad_val):
    res = []
    for index, seq in enumerate(data2d):
        if(len(seq) < max_len):
            res.append(np.concatenate([seq, np.full([max_len - len(seq)], pad_val)]))
        else:
            res.append(seq[:max_len])
    return res

def concat_list_h(list1, list2):
    res = []
    for i, ele in enumerate(list1):
        res.append(np.concatenate([ele, list2[i]]))
    return res



corpus = list(data['question1']) + list(data['question2'])

w2v_model = get_w2v(corpus[0:20000], 20, min_count = 2)


In [4]:
w2v_model.most_similar(positive=['高血压'])  

/home/dutir_2t/wangchenguang/anaconda3/envs/gpu-tf/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('高血脂', 0.9244159460067749),
 ('预防', 0.9213269352912903),
 ('心脏病', 0.9201167821884155),
 ('并发症', 0.9162768721580505),
 ('高血糖', 0.9144097566604614),
 ('八十岁', 0.9127856492996216),
 ('肾衰竭', 0.909296989440918),
 ('糖尿病', 0.9078564047813416),
 ('贫血', 0.9066025018692017),
 ('头重', 0.9045228362083435)]

In [5]:
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import time
import datetime  
from scipy.sparse import coo_matrix
import os
import sys

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES'] = "1,2"

class Elmo():
    def __init__(self, cnn_word_vec_size = 512, max_word_len = 30, max_seq_len = 20, learning_rate = 0.025, learning_min = 1e-4, 
                 n_epochs = 1, min_frequency = 2, batch_size = 8, conv_activate_fn = tf.nn.relu, 
                 lstm_hiden_size = 512,
                 filter_width_and_num = [(1,32),
                                         (2,32),
                                         (3,64),
                                         (4,128),
                                         (5,256),
                                         (6,512),
                                         (7,1024),]):
        self.cnn_word_vec_size = cnn_word_vec_size
        self.learning_rate = learning_rate
        self.n_epochs = n_epochs
        self.min_frequency = min_frequency
        self.max_word_len = max_word_len
        self.batch_size = batch_size
        self.filter_width_and_num = filter_width_and_num
        self.conv_activate_fn = conv_activate_fn
        self.max_seq_len = max_seq_len
        self.lstm_hiden_size = lstm_hiden_size
        self.graph = tf.Graph()
        
    def _exit(code, err_str):
        print(err_str)
        sys.exit(code)
        
    def _add_chars(self, word, char_list):
        for ch in list(word):
            if(ch not in char_list):
                char_list.append(ch)
    
    def _make_chars(self, splited_corpus):
        char_list = []
        for line in splited_corpus:
            for word in line.split():
                self._add_chars(word, char_list)
        return char_list, {word : i for (i, word) in enumerate(char_list)}
    
    def _onehot_encode(self, index, max_len):
        res = np.zeros([max_len])
        if(index >= 0):
            res[index] = 1
        return res
    
    def _make_chars_onehot(self, char_dic, char_onehot_dim):
        char_onehot_dic = {}
        return {ch : self._onehot_encode(char_dic[ch], char_onehot_dim) for ch in char_dic}
    
    def _make_chars_embedding_matrix(self, char_onehot_dic, char_onehot_dim):
        matrix = None
        for ch in char_onehot_dic:
            if(matrix is None):
                matrix = [char_onehot_dic[ch]]
            else:
                matrix = np.r_[matrix, [char_onehot_dic[ch]]]
        matrix = np.r_[matrix, [np.zeros(char_onehot_dim)]]
        return matrix
        
    def _make_word_char_ids(self, word, char_dic, padding_id, max_word_len):
        res = np.full([max_word_len], padding_id)
        for i, ch in enumerate(list(word)):
            if(ch not in char_dic):
                self._exit(1, "In word onehot: %s not in char_dic" % ch)
            if(i >= max_word_len):
                break
            res[i] = self.char_dic[ch]
        return res
    

    def _make_words_char_ids(self, splited_corpus, char_dic, max_word_len):
        words_char_ids = {}
        for line in splited_corpus:
            for word in line.split():
                if(word not in char_dic):
                    words_char_ids[word] = self._make_word_char_ids(word, char_dic,
                                            len(char_dic), max_word_len)
        return words_char_ids
    
    def _data_len(self, splited_corpus):
        max_len = -1
        sum_len = 0
        word_num = 0
        seq_max_len = -1
        seq_sum_len = 0
        seq_num = 0
        for line in splited_corpus:
            words = line.split()
            seq_num += 1
            seq_sum_len += len(words)
            if(len(words) > seq_max_len):
                seq_max_len = len(words)
            for word in words:
                word_num += 1
                sum_len += len(word)
                if(len(word) > max_len):
                    max_len = len(word)
        return max_len, sum_len / word_num, seq_max_len, seq_sum_len / seq_num
    
    def build_char_cnn(self, embedding_matrix):
        self.embedding_size = len(embedding_matrix[0])
        with tf.name_scope("elmo/char_cnn"), self.graph.as_default():
            self.input_word = tf.placeholder(shape = [None, self.max_word_len], 
                                             dtype = tf.int32, name = "input_word")
            with tf.device("/cpu:0"), tf.name_scope("embedding"):
                self.char_embedding_matrix_ = tf.Variable(embedding_matrix, dtype = tf.float32,
                                                         name = "char_embedding_matrix")
                self.word_embedded = tf.expand_dims(tf.nn.embedding_lookup(self.char_embedding_matrix_,
                                                                          self.input_word), -1, 
                                                    name = "word_embedded")
            with tf.name_scope("conv_maxpool"):
                self.pooled_outputs = []
                for filter_size, filter_num in self.filter_width_and_num:
                    with tf.name_scope("conv_%d_%d" % (filter_size, filter_num)):
                        filter_shape = [filter_size, self.embedding_size, 1, filter_num]
                        filter_weight = tf.Variable(tf.truncated_normal(filter_shape, stddev = 0.1), name = "filter_weight")
                        filter_bias = tf.Variable(tf.truncated_normal([filter_num], stddev = 0.1), name = "filter_bias")
                        conv_output = self.conv_activate_fn(tf.nn.bias_add(tf.nn.conv2d(
                            self.word_embedded,
                            filter_weight,
                            strides = [1, 1, 1, 1],
                            padding = 'VALID',
                        ), filter_bias, name = "conv_output"), name = "act_conv_output")
                    with tf.name_scope("maxpool_%d_%d" % (conv_output.shape[1], conv_output.shape[2])):
                        pooled_output = tf.nn.max_pool(conv_output,
                                                  ksize = [1, conv_output.shape[1], 1, 1],
                                                  strides = [1, 1, 1, 1],
                                                  padding = "VALID",
                                                  name = "pooled_output")
                        self.pooled_outputs.append(pooled_output)
                with tf.name_scope("concated_pool"):
                    self.conv_maxpool_output = tf.concat(self.pooled_outputs, -1)
                    self.conv_maxpool_output = tf.reshape(self.conv_maxpool_output, 
                                                         [-1, self.conv_maxpool_output.shape[-1]])
        with tf.name_scope("elmo/highway"), self.graph.as_default():
            hw_w_shape = [int(self.conv_maxpool_output.shape[1]),int(self.conv_maxpool_output.shape[1])]
            hw_b_shape = [int(self.conv_maxpool_output.shape[1])]
            self.carry_weight = tf.Variable(tf.random_normal(shape = hw_w_shape), name = 'carry_weight')
            self.carry_bias = tf.Variable(tf.random_normal(shape=hw_b_shape), name = 'carry_weight')
            self.transform_weight = tf.Variable(tf.random_normal(shape=hw_w_shape), name = 'carry_weight')
            self.transform_bias = tf.Variable(tf.random_normal(shape=hw_b_shape), name = 'carry_weight')
            self.carry_gate = tf.sigmoid(tf.nn.bias_add(tf.matmul(self.conv_maxpool_output, self.carry_weight), 
                                          self.carry_bias, name = 'carry_gate'))
            self.transform_gate = tf.nn.relu(tf.nn.bias_add(tf.matmul(self.conv_maxpool_output, self.transform_weight),
                                             self.transform_bias, name = 'transform_gate'))
            self.highway_output = self.transform_gate + (1.0 - self.carry_gate) * self.conv_maxpool_output
        
        with tf.name_scope("elmo/projection"), self.graph.as_default():
            self.project_output = tf.contrib.layers.fully_connected(self.highway_output, 
                                                              self.cnn_word_vec_size,
                                                              tf.nn.relu)
    
    def make_charcnn_wordvec(self):
        charcnn_wordvec = {}
        config = tf.ConfigProto()
        config.gpu_options.allow_growth = True
        config.allow_soft_placement = True
        with tf.Session(config = config, graph = self.graph) as self.cnn_sess:
            tf.global_variables_initializer().run()
            for word in self.words_char_ids:
                self.charcnn_wordvec[word] = self.project_output.eval(feed_dict = {
                    self.input_word : self.words_char_ids[word].reshape((-1, 8))
                })
        return charcnn_wordvec
                
    def build_bilstm(self):
        with tf.name_scope("elmo/bilstm"), self.graph.as_default():
            self.b_input_seqs = tf.placeholder(dtype = tf.float32,
                                              shape = [None, self.max_seq_len, self.cnn_word_vec_size])
            self.b_reverse_input_seqs = tf.placeholder(dtype = tf.float32,
                                              shape = [None, self.max_seq_len, self.cnn_word_vec_size])
            self.fw_lstm_cell = tf.contribib.rnn.LSTMCell(self.lstm_hiden_size)
            self.bw_lstm_cell = tf.contribib.rnn.LSTMCell(self.lstm_hiden_size)
            
    
    def fit(self, splited_corpus):
        print("Suggestion: real max word len is %d and mean of word len is %f real \
              max seq len is %d and mean of seq len is %f" % (self._data_len(splited_corpus)))
        print("Building chars...")
        self.char_list, self.char_dic = self._make_chars(splited_corpus)
        self.char_onehot_dim = len(self.char_list)
        print("Building chars onehot...")
        self.char_onehot_dic = self._make_chars_onehot(self.char_dic, self.char_onehot_dim)
        print("Building chars embedding matrix...")
        self.char_embedding_matrix = self._make_chars_embedding_matrix(self.char_onehot_dic,
                                                                      self.char_onehot_dim)
        print("Building words ids...")
        self.words_char_ids = self._make_words_char_ids(splited_corpus, self.char_dic, self.max_word_len)
        print("Building char_cnn model...")
        self.build_char_cnn(self.char_embedding_matrix)
        print("Building BiLSTM model...")
        self.build_bilstm()
        print("Building char_cnn word_vec...")
        self.charcnn_wordvec = self.make_charcnn_wordvec()
        
            
'''en_elmo = Elmo(5, n_epochs=200, learning_rate=0.01, max_word_len = 8)
en_elmo.fit(['king a','queen a','king b','queen b','king c','queen c','king x',
                            'queen y','man d','woman d','man e','woman e','man f','woman f',
                            'man x','woman y'])'''
cn_elmo = Elmo(cnn_word_vec_size = 512, n_epochs=200, learning_rate=0.01, max_word_len = 6, max_seq_len = 10)
cn_elmo.fit(corpus)
#cn_elmo.build(np.array(cn_elmo.char_embedding_matrix))

/home/dutir_2t/wangchenguang/anaconda3/envs/gpu-tf/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/dutir_2t/wangchenguang/anaconda3/envs/gpu-tf/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/dutir_2t/wangchenguang/anaconda3/envs/gpu-tf/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)]

Suggestion: real max word len is 27 and mean of word len is 1.926293 real               max seq len is 46 and mean of seq len is 6.678800
Building chars...
Building chars onehot...
Building chars embedding matrix...
Building words ids...
Building char_cnn model...
Building BiLSTM model...
Building char_cnn word_vec...


In [65]:
print(len(cn_elmo.char_embedding_matrix))

2161


In [97]:
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import time
import datetime  
from scipy.sparse import coo_matrix
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES'] = "0,2"

class Elmo():
    def __init__(self, word_vec_size = 512, max_char_len = 30, learning_rate = 0.025, learning_min = 1e-4, 
                 n_epochs = 1, min_frequency = 2, batch_size = 8):
        self.word_vec_size = word_vec_size
        self.learning_rate = learning_rate
        self.n_epochs = n_epochs
        self.min_frequency = min_frequency
        self.batch_size = batch_size
        self.graph = tf.Gtaph()
        
    def _add_chars()

    def _frequency_dict(self, words):
        dic = {}
        for line in words:
            for word in line:
                if(word in dic):
                    dic[word] += 1
                else:
                    dic[word] = 1
        return dic
    
    def _onehot_encode(self, index, max_len):
        res = np.zeros([max_len])
        if(index >= 0):
            res[index] = 1
        return res
    
    def _reduce(self, frequency_dict):
        if(self.min_frequency > 1):
            del_words = []
            for word in self.frequency_dict:
                if(self.frequency_dict[word] < self.min_frequency):
                    del_words.append(word)
            for dword in del_words:
                del self.frequency_dict[dword]
        return frequency_dict
   
    def make_data(self):
        self.mid_batches = []
        self.side_batches = []
        self.mid_label_batches = []
        self.side_label_batches = []
        if(self.model_type == 'cbow'):
            window_len = self.c_of_cbow
        elif(self.model_type == "skip-gram"):
            window_len = self.n_of_skgram
        else:
            print('In make data model type %s wrong' % (self.model_type))
        line_index = 0
        for line in self.words:
            if(line_index %1000 == 0):
                print('constructing window of line %d / %d' % (line_index, len(self.words)))
            line_index += 1
            for mid_index, mid_word in enumerate(line):
                window = self._extract_window(line, mid_index, window_len)
                mid, sides, mid_label, side_labels = self._extract_window_onehot(window, window_len)
                if(0 == len(sides)):
                    continue
                if(False == np.equal(mid, 0).all() and False == np.equal(sides, 0).all()):
                    if(self.model_type == "cbow"):
                        self.side_batches.append(coo_matrix(np.reshape(sides, (len(sides), -1))))
                        self.mid_batches.append(coo_matrix(np.reshape(mid, (1, -1))))
                        self.mid_label_batches.append(mid_label)
                    else:
                        for side_index, side in enumerate(sides):
                            self.side_batches.append(coo_matrix(np.reshape(side, (1, -1))))
                            self.mid_batches.append(coo_matrix(np.reshape(mid, (1, -1))))
                            self.side_label_batches.append(side_labels[side_index])
                        
    def fit(self, splited_corpus):
        self.words = [x.lower().split() for x in splited_corpus]
        self.frequency_dict = self._frequency_dict(self.words)
        self.frequency_dict = self._reduce(self.frequency_dict)
        self.frequency_dict = dict(sorted(self.frequency_dict.items(), 
                                     key = lambda d:d[1] , reverse = True))
        self.vocab = list(self.frequency_dict.keys())
        self.vocab_index = {word : i for (i, word) in enumerate(self.vocab)}
        self.vovab_size = len(self.vocab)
        print("constructing data...")
        self.make_data()      
        print('building...')
        self._build()
        print('ready for trainning...')
        self.sess = tf.Session()
        self.sess.run(tf.global_variables_initializer())

    
    def _build(self):
        with tf.name_scope("input"):
            self.x_input = tf.placeholder(dtype=tf.float32, shape = [None, self.vovab_size], name = "input_x")
            self.y_input = tf.placeholder(dtype=tf.float32, shape = [None, self.vovab_size], name = "input_y")
            self.y_label = tf.placeholder(dtype=tf.int32, shape = [None], name = "input_y_label")
        with tf.name_scope("fcl"):
            self.w2v_weight = tf.Variable(tf.random_uniform([self.vovab_size, self.w2v_size]),
                                     name = "w2v_weight")
            self.x_present = tf.matmul(self.x_input, self.w2v_weight)
            self.x_present = tf.reduce_mean(self.x_present, axis = 0)
            self.x_present = tf.expand_dims(self.x_present, 0)
            self.w2v_bias = tf.Variable(tf.random_uniform([self.vovab_size]),
                                     name = "w2v_bias")
        with tf.name_scope("output"):
            self.output_weight = tf.Variable(tf.random_uniform([self.w2v_size, self.vovab_size]),
                                     name = "w2v_weight")
            self.fcl_output = tf.matmul(self.x_present, self.output_weight, name = "output")
        with tf.name_scope("loss"):
            self.loss = tf.nn.nce_loss(
                weights=self.w2v_weight,
                biases=self.w2v_bias,
                inputs = self.x_present,
                labels = tf.expand_dims(self.y_label, 0),
                num_sampled=self.num_sampled,
                num_classes=self.y_input.shape[1]
            )
            '''self.loss = tf.reduce_mean(
                tf.nn.softmax_cross_entropy_with_logits(
                    logits = self.fcl_output,
                    labels = self.y_input
                )
            )'''
        self.optimizer = tf.train.GradientDescentOptimizer(learning_rate=self.learning_rate)
        self.train_step = self.optimizer.minimize(self.loss)
    
    def vec(self, word):
        if(word not in self.vocab):
            print('word %s not in vocab maybe frequency < %d' % (word, self.min_frequency))
            return
        else:
            oh = self._word_onehot(word).reshape(1, -1).astype(np.float32)
            return self.sess.run(tf.matmul(oh, self.w2v_weight))[0]
    
    def _cos(self, vec1, vec2): 
        return np.sum(vec1 * vec2) / (np.sqrt(np.sum(vec1 * vec1)) * np.sqrt(np.sum(vec2 * vec2)))
    
    def sim(self, word1, word2):
        if(word1 not in self.vocab):
            print('word %s not in vocab maybe frequency < %d' % (word1, self.min_frequency))
            return
        elif(word2 not in self.vocab):
            print('word %s not in vocab maybe frequency < %d' % (word2, self.min_frequency))
            return
        else:
            oh1 = self._word_onehot(word1).reshape(1, -1).astype(np.float32)
            oh2 = self._word_onehot(word2).reshape(1, -1).astype(np.float32)
            vec1 = self.sess.run(tf.matmul(oh1, self.w2v_weight))[0]
            vec2 = self.sess.run(tf.matmul(oh2, self.w2v_weight))[0]
            return self._cos(vec1, vec2)
    
    def check_words(self, words):
        for word in words:
            if(word not in self.vocab):
                print('word %s not in vocab maybe frequency < %d' % (word, self.min_frequency))
                return False
        return True
    
    def most_sim(self, pos_words, k, neg_words = []):
        if(False == self.check_words(pos_words) or False == self.check_words(neg_words)):
            return
        else:
            vec = None
            for pw in pos_words:
                oh = self._word_onehot(pw).reshape(1, -1).astype(np.float32)
                if(vec is None):
                    vec = self.sess.run(tf.matmul(oh, self.w2v_weight))[0]
                else:
                    vec += self.sess.run(tf.matmul(oh, self.w2v_weight))[0]
            for nw in neg_words:
                oh = self._word_onehot(nw).reshape(1, -1).astype(np.float32)
                if(vec is None):
                    vec = self.sess.run(tf.matmul(oh, self.w2v_weight))[0]
                else:
                    vec -= self.sess.run(tf.matmul(oh, self.w2v_weight))[0]
            a = vec
            b = self.sess.run(self.w2v_weight)
            sims = (np.sum(a * b, axis = 1).reshape(-1, 1)) / (np.sqrt(np.sum(b*b, axis = 1).reshape(-1, 1)) * np.sqrt(np.sum(a*a)))
            sims_dict = dict(zip(self.vocab, sims.reshape(1, -1).tolist()[0]))
            sims_dict = sorted(sims_dict.items(), key = lambda d:d[1] , reverse = True)
            for tup in sims_dict[:k+1]:
                print(tup)
            return sims_dict[:k+1]
    
    def _word_onehot(self, word):
        if(word == None or word not in self.vocab):
            return self._onehot_encode(-1, self.vovab_size)
        else:
            return self._onehot_encode(self.vocab_index[word], self.vovab_size)
    
    def _extract_window_onehot(self, window, window_side_len):
        if(window[window_side_len] not in self.vocab):
            return [], [], None, []
        mid = self._word_onehot(window[window_side_len])
        sides = []
        side_labels = []
        for word in window[:window_side_len] + window[window_side_len + 1:]:
            if(word != None and word in self.vocab):
                sides.append(self._word_onehot(word))
                side_labels.append(self.vocab_index[word])
        return mid, sides, self.vocab_index[window[window_side_len]], side_labels
    
    def _extract_window(self, line, mid_index, window_side_len):
        window = [None for i in range(2 * window_side_len + 1)]
        window_index = 0
        for index in range(mid_index - window_side_len , mid_index + window_side_len + 1):
            if(index < 0 or index >= len(line)):
                window_index += 1
                continue
            window[window_index] = line[index]
            window_index += 1
        return window
    
    def time_gap(self, old_time):
        if(old_time == None):
            return datetime.datetime.now(), None
        else:
            now_time = datetime.datetime.now()
            return now_time, (now_time - old_time).seconds
    def train_epoch(self, epoch):
        if(self.model_type == "cbow"):
            for batch_index, mid in enumerate(self.mid_batches):
                if(batch_index % 1000 == 0):
                    print('trainning epoch %d / %d  batch %d / %d' % (epoch + 1,self.n_epochs,
                                                                       batch_index, len(self.mid_batches)))
                feed_dict = {
                    self.x_input : self.side_batches[batch_index].toarray(),
                    self.y_input : mid.toarray(),
                    self.y_label : [self.mid_label_batches[batch_index]]
                }
                self.sess.run(self.train_step, feed_dict = feed_dict)
        elif(self.model_type == "skip-gram"):
            for batch_index, side in enumerate(self.side_batches):
                if(batch_index % 1000 == 0):
                    print('trainning epoch %d / %d  batch %d / %d' % (epoch,self.n_epochs,
                                                                       batch_index, len(self.mid_batches)))
                feed_dict = {
                    self.x_input : self.mid_batches[batch_index].toarray(),
                    self.y_input : side.toarray(),
                    self.y_label : [self.side_label_batches[batch_index]]
                }
                self.sess.run(self.train_step, feed_dict = feed_dict)
        else:
            print("train error model type wrong %s" % (self.model_type))
    def train(self):
        print("start trainning model %s" % (self.model_type))
        now_time = None
        for epoch in range(self.n_epochs):
            now_time, time_gap = self.time_gap(now_time)
            if(time_gap != None):
                print("##############epoch %d use %d seconds" % (epoch, time_gap))
            self.train_epoch(epoch)
            
            
            
m_w2v_model = Word2Vec(5, n_epochs=200, learning_rate=0.01, c_of_cbow=2, num_sampled=2, model_type= 'cbow')
m_w2v_model.fit(['king a','queen a','king b','queen b','king c','queen c','king x',
                            'queen y','man d','woman d','man e','woman e','man f','woman f',
                            'man x','woman y'])
m_w2v_model.train()
m_w2v_model.most_sim(pos_words=['king', 'woman'], neg_words=['man'], k=2)

SyntaxError: invalid syntax (<ipython-input-97-69e74dfb1852>, line 22)

In [47]:
cm_w2v_model.most_sim(pos_words=["鸡蛋"], k=10)

('鸡蛋', 1.0)
('冰糖', 0.9292140603065491)
('土豆', 0.9119759798049927)
('虾', 0.8807253837585449)
('猪肉', 0.8541760444641113)
('红薯', 0.8522436022758484)
('牛肉', 0.8506330847740173)
('开心果', 0.8435834646224976)
('菠萝', 0.8349963426589966)
('火龙果', 0.8239815831184387)
('海参', 0.8205407857894897)


[('鸡蛋', 1.0),
 ('冰糖', 0.9292140603065491),
 ('土豆', 0.9119759798049927),
 ('虾', 0.8807253837585449),
 ('猪肉', 0.8541760444641113),
 ('红薯', 0.8522436022758484),
 ('牛肉', 0.8506330847740173),
 ('开心果', 0.8435834646224976),
 ('菠萝', 0.8349963426589966),
 ('火龙果', 0.8239815831184387),
 ('海参', 0.8205407857894897)]

In [34]:
sm_w2v_model = Word2Vec(20, n_epochs=100, learning_rate=0.03, num_sampled = 15, model_type='skip-gram')
sm_w2v_model.fit(corpus[0:20000])
for epoch in range(100):
    sm_w2v_model.train_epoch(epoch)

constructing data...
constructing window of line 0 / 20000
constructing window of line 1000 / 20000
constructing window of line 2000 / 20000
constructing window of line 3000 / 20000
constructing window of line 4000 / 20000
constructing window of line 5000 / 20000
constructing window of line 6000 / 20000
constructing window of line 7000 / 20000
constructing window of line 8000 / 20000
constructing window of line 9000 / 20000
constructing window of line 10000 / 20000
constructing window of line 11000 / 20000
constructing window of line 12000 / 20000
constructing window of line 13000 / 20000
constructing window of line 14000 / 20000
constructing window of line 15000 / 20000
constructing window of line 16000 / 20000
constructing window of line 17000 / 20000
constructing window of line 18000 / 20000
constructing window of line 19000 / 20000
building...
ready for trainning...
trainning epoch 0 / 100  batch 0 / 521014
trainning epoch 0 / 100  batch 1000 / 521014
trainning epoch 0 / 100  batch

trainning epoch 0 / 100  batch 158000 / 521014
trainning epoch 0 / 100  batch 159000 / 521014
trainning epoch 0 / 100  batch 160000 / 521014
trainning epoch 0 / 100  batch 161000 / 521014
trainning epoch 0 / 100  batch 162000 / 521014
trainning epoch 0 / 100  batch 163000 / 521014
trainning epoch 0 / 100  batch 164000 / 521014
trainning epoch 0 / 100  batch 165000 / 521014
trainning epoch 0 / 100  batch 166000 / 521014
trainning epoch 0 / 100  batch 167000 / 521014
trainning epoch 0 / 100  batch 168000 / 521014
trainning epoch 0 / 100  batch 169000 / 521014
trainning epoch 0 / 100  batch 170000 / 521014
trainning epoch 0 / 100  batch 171000 / 521014
trainning epoch 0 / 100  batch 172000 / 521014
trainning epoch 0 / 100  batch 173000 / 521014
trainning epoch 0 / 100  batch 174000 / 521014
trainning epoch 0 / 100  batch 175000 / 521014
trainning epoch 0 / 100  batch 176000 / 521014
trainning epoch 0 / 100  batch 177000 / 521014
trainning epoch 0 / 100  batch 178000 / 521014
trainning epo

trainning epoch 0 / 100  batch 333000 / 521014
trainning epoch 0 / 100  batch 334000 / 521014
trainning epoch 0 / 100  batch 335000 / 521014
trainning epoch 0 / 100  batch 336000 / 521014
trainning epoch 0 / 100  batch 337000 / 521014
trainning epoch 0 / 100  batch 338000 / 521014
trainning epoch 0 / 100  batch 339000 / 521014
trainning epoch 0 / 100  batch 340000 / 521014
trainning epoch 0 / 100  batch 341000 / 521014
trainning epoch 0 / 100  batch 342000 / 521014
trainning epoch 0 / 100  batch 343000 / 521014
trainning epoch 0 / 100  batch 344000 / 521014
trainning epoch 0 / 100  batch 345000 / 521014
trainning epoch 0 / 100  batch 346000 / 521014
trainning epoch 0 / 100  batch 347000 / 521014
trainning epoch 0 / 100  batch 348000 / 521014
trainning epoch 0 / 100  batch 349000 / 521014
trainning epoch 0 / 100  batch 350000 / 521014
trainning epoch 0 / 100  batch 351000 / 521014
trainning epoch 0 / 100  batch 352000 / 521014
trainning epoch 0 / 100  batch 353000 / 521014
trainning epo

trainning epoch 0 / 100  batch 508000 / 521014
trainning epoch 0 / 100  batch 509000 / 521014
trainning epoch 0 / 100  batch 510000 / 521014
trainning epoch 0 / 100  batch 511000 / 521014
trainning epoch 0 / 100  batch 512000 / 521014
trainning epoch 0 / 100  batch 513000 / 521014
trainning epoch 0 / 100  batch 514000 / 521014
trainning epoch 0 / 100  batch 515000 / 521014
trainning epoch 0 / 100  batch 516000 / 521014
trainning epoch 0 / 100  batch 517000 / 521014
trainning epoch 0 / 100  batch 518000 / 521014
trainning epoch 0 / 100  batch 519000 / 521014
trainning epoch 0 / 100  batch 520000 / 521014
trainning epoch 0 / 100  batch 521000 / 521014
trainning epoch 1 / 100  batch 0 / 521014
trainning epoch 1 / 100  batch 1000 / 521014
trainning epoch 1 / 100  batch 2000 / 521014
trainning epoch 1 / 100  batch 3000 / 521014
trainning epoch 1 / 100  batch 4000 / 521014
trainning epoch 1 / 100  batch 5000 / 521014
trainning epoch 1 / 100  batch 6000 / 521014
trainning epoch 1 / 100  batch

trainning epoch 1 / 100  batch 163000 / 521014
trainning epoch 1 / 100  batch 164000 / 521014
trainning epoch 1 / 100  batch 165000 / 521014
trainning epoch 1 / 100  batch 166000 / 521014
trainning epoch 1 / 100  batch 167000 / 521014
trainning epoch 1 / 100  batch 168000 / 521014
trainning epoch 1 / 100  batch 169000 / 521014
trainning epoch 1 / 100  batch 170000 / 521014
trainning epoch 1 / 100  batch 171000 / 521014
trainning epoch 1 / 100  batch 172000 / 521014
trainning epoch 1 / 100  batch 173000 / 521014
trainning epoch 1 / 100  batch 174000 / 521014
trainning epoch 1 / 100  batch 175000 / 521014
trainning epoch 1 / 100  batch 176000 / 521014
trainning epoch 1 / 100  batch 177000 / 521014
trainning epoch 1 / 100  batch 178000 / 521014
trainning epoch 1 / 100  batch 179000 / 521014
trainning epoch 1 / 100  batch 180000 / 521014
trainning epoch 1 / 100  batch 181000 / 521014
trainning epoch 1 / 100  batch 182000 / 521014
trainning epoch 1 / 100  batch 183000 / 521014
trainning epo

...

trainning epoch 11 / 100  batch 369000 / 521014
trainning epoch 11 / 100  batch 370000 / 521014
trainning epoch 11 / 100  batch 371000 / 521014
trainning epoch 11 / 100  batch 372000 / 521014
trainning epoch 11 / 100  batch 373000 / 521014
trainning epoch 11 / 100  batch 374000 / 521014
trainning epoch 11 / 100  batch 375000 / 521014
trainning epoch 11 / 100  batch 376000 / 521014
trainning epoch 11 / 100  batch 377000 / 521014
trainning epoch 11 / 100  batch 378000 / 521014
trainning epoch 11 / 100  batch 379000 / 521014
trainning epoch 11 / 100  batch 380000 / 521014
trainning epoch 11 / 100  batch 381000 / 521014
trainning epoch 11 / 100  batch 382000 / 521014
trainning epoch 11 / 100  batch 383000 / 521014
trainning epoch 11 / 100  batch 384000 / 521014
trainning epoch 11 / 100  batch 385000 / 521014
trainning epoch 11 / 100  batch 386000 / 521014
trainning epoch 11 / 100  batch 387000 / 521014
trainning epoch 11 / 100  batch 388000 / 521014
trainning epoch 11 / 100  batch 389000 /

trainning epoch 12 / 100  batch 19000 / 521014
trainning epoch 12 / 100  batch 20000 / 521014
trainning epoch 12 / 100  batch 21000 / 521014
trainning epoch 12 / 100  batch 22000 / 521014
trainning epoch 12 / 100  batch 23000 / 521014
trainning epoch 12 / 100  batch 24000 / 521014
trainning epoch 12 / 100  batch 25000 / 521014
trainning epoch 12 / 100  batch 26000 / 521014
trainning epoch 12 / 100  batch 27000 / 521014
trainning epoch 12 / 100  batch 28000 / 521014
trainning epoch 12 / 100  batch 29000 / 521014
trainning epoch 12 / 100  batch 30000 / 521014
trainning epoch 12 / 100  batch 31000 / 521014
trainning epoch 12 / 100  batch 32000 / 521014
trainning epoch 12 / 100  batch 33000 / 521014
trainning epoch 12 / 100  batch 34000 / 521014
trainning epoch 12 / 100  batch 35000 / 521014
trainning epoch 12 / 100  batch 36000 / 521014
trainning epoch 12 / 100  batch 37000 / 521014
trainning epoch 12 / 100  batch 38000 / 521014
trainning epoch 12 / 100  batch 39000 / 521014
trainning epo

trainning epoch 12 / 100  batch 192000 / 521014
trainning epoch 12 / 100  batch 193000 / 521014
trainning epoch 12 / 100  batch 194000 / 521014
trainning epoch 12 / 100  batch 195000 / 521014
trainning epoch 12 / 100  batch 196000 / 521014
trainning epoch 12 / 100  batch 197000 / 521014
trainning epoch 12 / 100  batch 198000 / 521014
trainning epoch 12 / 100  batch 199000 / 521014
trainning epoch 12 / 100  batch 200000 / 521014
trainning epoch 12 / 100  batch 201000 / 521014
trainning epoch 12 / 100  batch 202000 / 521014
trainning epoch 12 / 100  batch 203000 / 521014
trainning epoch 12 / 100  batch 204000 / 521014
trainning epoch 12 / 100  batch 205000 / 521014
trainning epoch 12 / 100  batch 206000 / 521014
trainning epoch 12 / 100  batch 207000 / 521014
trainning epoch 12 / 100  batch 208000 / 521014
trainning epoch 12 / 100  batch 209000 / 521014
trainning epoch 12 / 100  batch 210000 / 521014
trainning epoch 12 / 100  batch 211000 / 521014
trainning epoch 12 / 100  batch 212000 /

trainning epoch 12 / 100  batch 363000 / 521014
trainning epoch 12 / 100  batch 364000 / 521014
trainning epoch 12 / 100  batch 365000 / 521014
trainning epoch 12 / 100  batch 366000 / 521014
trainning epoch 12 / 100  batch 367000 / 521014
trainning epoch 12 / 100  batch 368000 / 521014
trainning epoch 12 / 100  batch 369000 / 521014
trainning epoch 12 / 100  batch 370000 / 521014
trainning epoch 12 / 100  batch 371000 / 521014
trainning epoch 12 / 100  batch 372000 / 521014
trainning epoch 12 / 100  batch 373000 / 521014
trainning epoch 12 / 100  batch 374000 / 521014
trainning epoch 12 / 100  batch 375000 / 521014
trainning epoch 12 / 100  batch 376000 / 521014
trainning epoch 12 / 100  batch 377000 / 521014
trainning epoch 12 / 100  batch 378000 / 521014
trainning epoch 12 / 100  batch 379000 / 521014
trainning epoch 12 / 100  batch 380000 / 521014
trainning epoch 12 / 100  batch 381000 / 521014
trainning epoch 12 / 100  batch 382000 / 521014
trainning epoch 12 / 100  batch 383000 /

KeyboardInterrupt: 

In [48]:
sm_w2v_model.most_sim(pos_words=["鸡蛋"], k=10)

('鸡蛋', 0.9999998807907104)
('土豆', 0.8679536581039429)
('黄豆', 0.8657839894294739)
('红薯', 0.8618475794792175)
('木瓜', 0.8338364958763123)
('藕', 0.8316220045089722)
('香蕉', 0.8275298476219177)
('燕窝', 0.8251843452453613)
('榴莲', 0.823262095451355)
('西洋参', 0.8115437626838684)
('花生米', 0.8102536797523499)


[('鸡蛋', 0.9999998807907104),
 ('土豆', 0.8679536581039429),
 ('黄豆', 0.8657839894294739),
 ('红薯', 0.8618475794792175),
 ('木瓜', 0.8338364958763123),
 ('藕', 0.8316220045089722),
 ('香蕉', 0.8275298476219177),
 ('燕窝', 0.8251843452453613),
 ('榴莲', 0.823262095451355),
 ('西洋参', 0.8115437626838684),
 ('花生米', 0.8102536797523499)]